In [1]:
import re
import json

# Pattern large pour retirer tout le préfixe commençant par "ترجم" jusqu'au premier deux-points.
prefix_pattern = r"^\s*ترجم(?:\s+من\s+[^:]+)?\s*:\s*"

def clean_prefix(text):
    """ Retire le préfixe et remplace "\xa0" par un espace. """
    cleaned = re.sub(prefix_pattern, "", text)
    cleaned = cleaned.replace("\xa0", " ").strip()
    print(f"clean_prefix:\n  Original: {text}\n  Nettoyé: {cleaned}")
    return cleaned

def split_conversation(convo):
    """
    Découpe une chaîne de conversation multi-tours contenant les marqueurs
    "<|assistant|>" et "<|user|>" en une liste de paires {texte_cible, traduction}.
    
    Même si un segment ne contient pas "<|assistant|>", il est ajouté avec traduction vide.
    """
    print("\n--- split_conversation ---")
    print("Convo brute :", convo)
    pairs = []
    segments = convo.split("<|user|>")
    print(f"Segments obtenus (nombre={len(segments)}):")
    for idx, seg in enumerate(segments):
        print(f"Segment {idx+1} brut: {seg}")
    for seg in segments:
        seg = seg.strip()
        if not seg:
            continue
        if "<|assistant|>" in seg:
            parts = seg.split("<|assistant|>")
            user_part = parts[0].strip()
            assistant_part = parts[1].strip() if len(parts) > 1 else ""
        else:
            user_part = seg
            assistant_part = ""
        # Nettoyer le préfixe avec clean_prefix
        user_part = clean_prefix(user_part)
        # Nettoyage complémentaire : retirer les marqueurs et retours de ligne
        user_part = user_part.replace("\\n", "").replace("<|assistant|>", "").replace("<|user|>", "").strip()
        assistant_part = assistant_part.replace("\\n", "").replace("<|assistant|>", "").replace("<|user|>", "").strip()
        print(f"Segment nettoyé: texte_cible='{user_part}', traduction='{assistant_part}'")
        pairs.append({"texte_cible": user_part, "traduction": assistant_part})
    print("Pairs obtenues dans split_conversation:", pairs)
    return pairs

def extract_pairs(row_str):
    """
    Extrait les paires depuis une chaîne représentant plusieurs Row.
    
    Si le premier Row utilisateur contient des marqueurs multi-tours,
    on utilise split_conversation sur son contenu et, si le dernier segment a
    une traduction vide, on le complète avec le contenu du dernier Row assistant.
    """
    print("\n--- extract_pairs ---")
    print("Row_str brut:", row_str)
    if not isinstance(row_str, str):
        print("Erreur : l'entrée n'est pas une chaîne de caractères.")
        return []
    
    try:
        pattern = r"Row\(content=(?P<quote>['\"])(?P<content>.*?)(?P=quote),\s*role=['\"](?P<role>.*?)['\"]\)"
        messages = re.findall(pattern, row_str, re.DOTALL)
        print("Messages extraits:", messages)
    except Exception as e:
        print("Erreur lors de l'extraction avec re.findall:", e)
        return []
    
    if not messages:
        print("Aucun message extrait.")
        return []
    
    # Si le premier Row (utilisateur) contient des marqueurs, utiliser split_conversation.
    if messages[0][2] == "user" and ("<|assistant|>" in messages[0][1] or "<|user|>" in messages[0][1]):
        user_content = messages[0][1]
        external_assistant = ""
        for tup in reversed(messages):
            if tup[2] == "assistant":
                external_assistant = tup[1].strip()
                break
        print("Utilisation de split_conversation sur le contenu:", user_content)
        pairs = split_conversation(user_content)
        # Compléter le dernier pair s'il n'a pas de traduction avec external_assistant.
        if pairs and pairs[-1]['traduction'] == "" and external_assistant:
            print("Complétion du dernier pair avec external_assistant:", external_assistant)
            pairs[-1]['traduction'] = external_assistant
        print("Pairs extraites via split_conversation:", pairs)
        return pairs
    
    # Méthode standard (alternance)
    msg_list = []
    for tup in messages:
        try:
            quote, content, role = tup[0], tup[1], tup[2]
            if "<|assistant|>" in content or "<|user|>" in content:
                clean_content = content.strip()
            else:
                clean_content = content.replace("\\n", "").strip()
            msg_list.append({"role": role, "content": clean_content})
        except Exception as e:
            print("Erreur lors du traitement d'un message:", e)
    
    for msg in msg_list:
        if msg.get("role") == "user":
            if "<|assistant|>" not in msg["content"] and "<|user|>" not in msg["content"]:
                try:
                    msg["content"] = clean_prefix(msg["content"])
                    msg["content"] = msg["content"].replace("\\n", "").replace("\xa0", " ").strip()
                except Exception as e:
                    print("Erreur lors du nettoyage du message utilisateur:", e)
    
    pairs = []
    i = 0
    while i < len(msg_list):
        if msg_list[i].get("role") == "user":
            user_text = msg_list[i].get("content", "")
            assistant_text = ""
            if i + 1 < len(msg_list) and msg_list[i+1].get("role") == "assistant":
                assistant_text = msg_list[i+1].get("content", "")
            else:
                print("Attention: Pas de message assistant pour le message utilisateur:", user_text)
            pairs.append({"texte_cible": user_text, "traduction": assistant_text})
            i += 2
        else:
            i += 1
    print("Pairs extraites (méthode standard):", pairs)
    return pairs

def safe_extract(row):
    """
    Extrait les paires à partir de la colonne 'messages' d'une ligne.
    Si le dernier message utilisateur est sans réponse, il est complété par la traduction externe.
    """
    print("\n--- safe_extract ---")
    try:
        pairs = extract_pairs(row['messages'])
        user_count = row['messages'].count("<|user|>") + 1  # Le premier message n'a pas de marqueur
        assistant_count = row['messages'].count("<|assistant|>")
        print(f"User_count: {user_count}, Assistant_count: {assistant_count}")
        if user_count - assistant_count == 1 and pairs:
            if pairs[-1]['traduction'] == "":
                print("Complétion du dernier pair avec traduction externe:", row.get("traduction", "").strip())
                pairs[-1]['traduction'] = row.get("traduction", "").strip()
        print("Pairs finales dans safe_extract:", pairs)
        return pairs
    except Exception as e:
        print("Erreur dans safe_extract:", e)
        return []

# Exemple d'input pour tester avec Flores+_few_shot_366
test_row = {
    'messages': """[Row(content="ترجم من الفرنساوية للدارجة:
Il a récemment perdu un match contre Raonic durant l'Open de Brisbane.
<|assistant>
راه خسر هاذ الايامات ضد راونيتش في بطولة بريسبان المفتوحة.
<|user>
ترجم من الفرنساوية للدارجة:
Rejoindre un tel réseau ne nécessite généralement que de remplir un formulaire en ligne\xa0; bien que certains réseaux proposent ou nécessitent une vérification complémentaire.
<|assistant>
الانضمام لهاد الشبكة تيتطلب ملئ استمارة عبر الانترنت وصاف؛ واخا شي شبكات تيعطي أو تيطلب إثباتات إضافية.
<|user>
ترجم من الفرنساوية للدارجة:
Lorsque les individus présentent plusieurs variantes d'un trait particulier, ils sont polymorphes.
<|assistant>
فاش شي ناس كيكون عندهم بزاف ديال النسخ من شي صِفه معينه فيهم، فراه كيتسماو بولومورفيك.
<|user>
ترجم من الفرنساوية للدارجة:
On estime que le typhon se dirige vers la Chine à une vitesse de 11\xa0km/h.", role='user'), Row(content='كتشير التقديرات باللي الإعصار كيتحرك نحو الصين بسرعة حْداش كيلومتر فالساعة/ 11 كم / ساعة.', role='assistant')]""",
    'traduction': ""  # Ici, la traduction externe est utilisée pour compléter le dernier segment, mais dans ce cas le Row assistant externe sera utilisé.
}

# Test de safe_extract avec le test_row
result = safe_extract(test_row)
print("\n=== Résultat final ===")
print(json.dumps(result, ensure_ascii=False, indent=4))



--- safe_extract ---

--- extract_pairs ---
Row_str brut: [Row(content="ترجم من الفرنساوية للدارجة:
Il a récemment perdu un match contre Raonic durant l'Open de Brisbane.
<|assistant>
راه خسر هاذ الايامات ضد راونيتش في بطولة بريسبان المفتوحة.
<|user>
ترجم من الفرنساوية للدارجة:
Rejoindre un tel réseau ne nécessite généralement que de remplir un formulaire en ligne ; bien que certains réseaux proposent ou nécessitent une vérification complémentaire.
<|assistant>
الانضمام لهاد الشبكة تيتطلب ملئ استمارة عبر الانترنت وصاف؛ واخا شي شبكات تيعطي أو تيطلب إثباتات إضافية.
<|user>
ترجم من الفرنساوية للدارجة:
Lorsque les individus présentent plusieurs variantes d'un trait particulier, ils sont polymorphes.
<|assistant>
فاش شي ناس كيكون عندهم بزاف ديال النسخ من شي صِفه معينه فيهم، فراه كيتسماو بولومورفيك.
<|user>
ترجم من الفرنساوية للدارجة:
On estime que le typhon se dirige vers la Chine à une vitesse de 11 km/h.", role='user'), Row(content='كتشير التقديرات باللي الإعصار كيتحرك نحو الصين بسرعة حْ